- Gabriel Quiroz 19255
- Jose Pablo Ponce 19092

In [25]:
#Importar las librerías

import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz

## Parte 1 – Ingeniería de características


### Exploración de datos

1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.


In [26]:
## Cargar el dataset proporcionado
df = pd.read_csv('dataset_pishing.csv')
print(df.shape)
pd.set_option('display.max_columns', None)
df.sample(n=5).head() 

(11430, 67)


,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
7718,https://7426fbe0d8676fde2cac756c0731ce57.udagw...,1,0,0,0,0,0,0,1,1,1,2,0,0,0,0,0,0,4,2,6,12,6,75,32,75,31.25,22.0,40.500000,1,0,0,0,0,0,50,0.540000,0.460000,0,1,0,0.347826,0,0.565217,1,1,64.285714,0,0.0,100.0,0,0,0,41.666667,0,0,0,1,1,0,0,-1,0,0,1,0,phishing
1368,https://comunicazioniarubait.tumblr.com/ARU?qs...,1,0,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0,5,8,2,6,2,89,20,89,24.00,13.0,31.333333,0,1,0,0,0,0,0,0.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.0,0.0,0,0,0,0.000000,0,0,0,1,0,0,1050,5158,0,0,1,8,phishing
1550,http://www.haroldwashingtondemocraticclub.org/...,0,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,6,4,3,3,3,30,30,7,8.50,16.5,4.500000,0,0,0,0,0,0,9,0.777778,0.222222,0,0,0,0.000000,0,1.000000,0,0,0.000000,0,0.0,100.0,0,0,0,100.000000,0,0,0,1,1,0,32,-1,0,0,1,0,phishing
10980,https://www.pakissan.com/,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,4,3,3,0,8,8,0,5.50,5.5,0.000000,0,0,0,0,0,0,45,0.555556,0.444444,0,4,0,0.000000,0,0.050000,0,1,54.166667,0,0.0,0.0,0,0,0,75.000000,0,0,0,0,0,0,215,7090,354959,0,0,3,legitimate
87,https://www.mummyandmini.com/,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,4,3,3,0,12,12,0,7.50,7.5,0.000000,0,0,0,0,0,0,81,0.691358,0.308642,0,0,0,0.120000,0,0.000000,0,0,58.333333,0,0.0,0.0,0,0,0,20.000000,0,0,0,1,1,0,27,1800,0,0,0,3,legitimate


2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como
“pishing”. ¿Está balanceado el dataset?

In [27]:
print('legitimate:', len(df[df['status']=='legitimate']))
print('phising:', len(df[df['status']=='phishing']))

legitimate: 5715
phising: 5715


Como se puede observar hay la misma cantidad de observaciones tanto para "legitimate" y "phising", por ende se puede afirmar que el dataset esta balanceado

### Derivación de características


En base al artículo “Towards Benchmark Datasets for ML Based Wensite Phishing Detection: An
Experimental Study”, derivar las características basadas en el dominio: f1, f2, f4 – f20, f25, f26 y f27.

In [28]:
from urllib.parse import urlparse
def get_hostname(url):
    o = urlparse(url)
    return o.hostname
    
df['hostname'] = df['url'].apply(get_hostname)


In [29]:
#f1
df['f1'] = df['url'].str.len()


In [30]:

#f2
df['f2'] = df['hostname'].str.len()


In [31]:
#f4
df['f4'] = df['url'].str.count('\\.')

#f5
df['f5'] = df['url'].str.count('\\-')

#f6
df['f6'] = df['url'].str.count('\\@')

#f7
df['f7'] = df['url'].str.count('\\?')

#f8
df['f8'] = df['url'].str.count('\\&')

#f9
df['f9'] = df['url'].str.count('\\|')

#f10
df['f10'] = df['url'].str.count('\\=')

#f11
df['f11'] = df['url'].str.count('\\_')

#f12
df['f12'] = df['url'].str.count('\\˜')

#f13
df['f13'] = df['url'].str.count('\\%')

#f14
df['f14'] = df['url'].str.count('\\/')

#f15
df['f15'] = df['url'].str.count('\\*')

#f16
df['f16'] = df['url'].str.count('\\:')

#f17
df['f17'] = df['url'].str.count('\\,')

#f18
df['f18'] = df['url'].str.count('\\;')

#f19
df['f19'] = df['url'].str.count('\\$')

#f20
df['f20'] = df['url'].str.count('\\%20')


In [32]:
#f25
df['f25'] = df['url'].str.startswith('https')
df["f25"] = df["f25"].astype(int)


In [33]:
#f26
def count_digits(url):
    digits = sum(c.isdigit() for c in url)
    url_len = len(url)
    return digits / url_len

df["f26"] = df["url"].apply(count_digits)

In [34]:
#f27
df["f27"] = df["hostname"].apply(count_digits)

In [35]:
df.sample(n=5).head() 

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status,hostname,f1,f2,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f25,f26,f27
6043,http://in-medias-res.it/wp-includes/js/jcrop/p...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3,0,9,1,2,2,2,8,6,8,4.444444,3.666667,4.833333,4,0,0,0,0,0,12,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,100.000000,0.000000,0,0,0,100.000000,0,0,0,1,0,0,335,1857,0,0,1,0,phishing,in-medias-res.it,58,16,1,3,0,0,0,0,0,0,0,0,7,0,1,0,0,0,0,0,0.000000,0.000000
1487,http://www.nascar.com/buy-tickets/,0,1,0,0,0,0,0,0,0,0,2,1,0,0,0,1,0,4,3,3,3,3,7,6,7,4.750000,4.500000,5.000000,0,0,0,0,0,0,108,0.416667,0.583333,0,4,0,0.142857,0,0.031746,0,1,35.714286,0,63.636364,36.363636,0,0,0,12.500000,0,0,0,0,0,0,522,8973,11722,0,0,6,legitimate,www.nascar.com,34,14,2,1,0,0,0,0,0,0,0,0,4,0,1,0,0,0,0,0,0.000000,0.000000
7861,http://173.199.186.15/qabdlshop/libraries/joom...,1,0,0,0,0,0,0,1,1,1,3,0,0,0,0,0,0,14,3,2,2,2,11,3,11,4.928571,2.750000,5.800000,1,0,0,0,0,2,0,0.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0,0,0,0.000000,0,0,1,1,0,1,0,-1,0,1,1,0,phishing,173.199.186.15,75,14,3,0,0,0,0,0,0,0,0,0,9,0,1,0,0,0,0,0,0.146667,0.785714
1729,http://www.healthdisasteroc.org/,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,2,3,3,3,0,16,16,0,9.500000,9.500000,0.000000,0,0,0,0,0,0,147,0.904762,0.095238,0,0,0,0.571429,0,0.071429,0,0,92.857143,0,100.000000,0.000000,0,0,0,29.411765,0,0,0,1,0,0,1513,4330,1581987,0,0,4,legitimate,www.healthdisasteroc.org,32,24,2,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0.000000,0.000000
2309,https://persediaan.jogjaprov.go.id/wp-wp/Owa/i...,0,0,0,0,0,0,0,0,0,0,3,1,1,0,0,0,0,8,2,2,9,2,10,10,5,4.625000,9.500000,3.000000,2,0,0,0,1,0,20,0.150000,0.850000,0,2,0,0.000000,0,0.000000,0,1,0.000000,0,100.000000,0.000000,0,0,0,0.000000,0,0,0,1,0,0,545,4914,58891,0,1,4,phishing,persediaan.jogjaprov.go.id,59,26,4,1,0,1,0,0,1,0,0,0,5,0,1,0,0,0,0,1,0.000000,0.000000


### Preprocesamiento

Realice las modificaciones necesarias para convertir la variable categórica status a una variable
binaria. Elimine la columna del dominio.

In [36]:
df['status'] = df['status'].map({'phishing': 1, 'legitimate': 0})


In [37]:
df = df.drop(['hostname'], axis=1)

### Visualización de resultados


Genere un reporte de perfil con la librería pandas_profiling. Analice el reporte y determine las
columnas que son constantes, o que no tienen una varianza alta con la columna status. Almacene su
reporte como una página html.

In [38]:
from pandas_profiling import ProfileReport

In [39]:
prof = ProfileReport(df)
prof.to_file(output_file='reporte.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00,  3.79it/s]


### Selección de Características


En base al análisis del reporte, elimine las características repetidas o irrelevantes para la clasificación
de un sitio de pishing. Verifique que no posee observaciones repetidas.

# mira que podes eliminar que sea irrelevante ------------------------

In [128]:
df = df.drop(['domain_registration_length', 'sfh', 'submit_email', 'ratio_extErrors', 'ratio_intErrors','ratio_extRedirection', 'ratio_intRedirection', 'ratio_nullHyperlinks', 'nb_redirection', 'f9', 'f12', ], axis=1)

In [129]:
print('Duplicados antes de limpieza', df.duplicated().sum())
df = df.drop_duplicates()
print('Duplicados luego de limpieza', df.duplicated().sum())


Duplicados antes de limpieza 204
Duplicados luego de limpieza 0


## Parte 2 – Implementación


### Separación de datos


- Datos de entrenamiento: 55%
- Datos de validación: 15%
- Datos de prueba: 30%
- Almacene cada dataset como un archivo .csv